# Practical 5: Modelling & Evaluation
### Prepared by: <a href="https://www.linkedin.com/in/a-kanaan/">Dr Abdulkarim M. Jamal Kanaan</a> 
<hr>

* Acknowledgements: I would like to acknowledge the book "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow" written by Aurélien Géron. This practical exercise was heavily influenced by Chapter 2 of the book, titled "End-to-End Machine Learning Project." 

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/a-kanaan/dm-practicals/blob/main/practical4/practical4_data-preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

Here are the some of the steps for modelling (short-list promising models):

* If you are dealing with a large dataset, it may be beneficial to sample smaller training sets in order to train multiple models within a reasonable timeframe. However, it's important to note that this approach may penalize complex models like large neural networks or Random Forests.
* Once again, it is recommended to automate these steps as much as possible.
1. Start by training several quick and rough models from different categories, such as linear, naive Bayes, SVM, Random Forests, neural networks, etc., using standard parameters.
2. Measure and compare the performance of each model.
    - Utilize K-fold cross-validation for each model and calculate the mean and standard deviation of their performance.
3. Analyze the most significant variables for each algorithm.
4. Examine the types of errors that the models make.
    - Consider what additional data a human might have used to avoid these errors.
5. Conduct a quick round of feature selection and engineering.
6. Repeat the previous five steps one or two more times.
7. Narrow down the selection to the top three to five most promising models, giving preference to models that make different types of errors.
   
adapted from: https://github.com/ageron/handson-ml3/blob/main/ml-project-checklist.md

## Loading the dataset

In [8]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
sklearn.set_config(display="diagram")

In [11]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [12]:
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

print("Length of train_set:", len(train_set))
print("Length of test_set:", len(test_set))

Length of train_set: 16512
Length of test_set: 4128


In [ ]:
train_set["rooms_per_house"] = train_set["total_rooms"] / train_set["households"]
train_set["bedrooms_ratio"] = train_set["total_bedrooms"] / train_set["total_rooms"]
train_set["people_per_house"] = train_set["population"] / train_set["households"]

In [13]:
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

## Last Preprocessing

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]

num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"), 
    # you may add log transformation if needed
    StandardScaler()
)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [15]:
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared

array([[ 1.17299302, -1.35041487,  0.42853749, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.26802809, -1.37853628, -1.47350948, ...,  0.        ,
         0.        ,  1.        ],
       [-1.3529389 ,  0.98834939, -0.04697426, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.11760365,  0.30406165, -0.99799774, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18799856, -0.72705686, -0.522486  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.35269038, -0.66144022, -0.522486  , ...,  0.        ,
         0.        ,  0.        ]])

## Select and Train a Model on the Training Set

### Linear Regression

Fortunately, due to the progress made in the previous steps, the upcoming tasks will be straightforward. To begin, you opt to train a simple linear regression model as a starting point.

In [16]:
from sklearn.linear_model import LinearRegression

lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ocean_proximity'])])),
                ('linearregression', LinearRegression())])

In [17]:
housing_predictions = lin_reg.predict(housing) #y_predit, y_hat   housing is training set
housing_predictions[:5].round(-2)  # -2 = rounded to the nearest hundred

array([288000., 227300., 291900., 265800.,  21800.])

In [18]:
housing_labels.iloc[:5].values #housing_labels,y_true,observed y

array([291000., 156100., 353900., 241200.,  53800.])

In [19]:
# extra code – computes the error ratios
error_ratios = housing_predictions[:5].round(-2) / housing_labels.iloc[:5].values -1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

-1.0%, 45.6%, -17.5%, 10.2%, -59.5%


In [20]:
from sklearn.metrics import mean_squared_error

lin_rmse = mean_squared_error(housing_labels, 
                              housing_predictions,
                              squared=False) #squared : If True returns MSE value, if False returns RMSE value.
lin_rmse  #training error
#underfitting - training error high & validation error high  cause: using simple model , unrelevant feature used and too much retrictions

68458.0127473328

### Decision Tree Regressor

In [21]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ocean_proximity'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [25]:
tree_reg.fit(housing, housing_labels)
housing_predictions = tree_reg.predict(housing)
tree_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
tree_rmse # likely overfitting - training error low & validation error high - cause: small training set, unrelevant features used and less retrictions

0.0

### (K-fold) Cross-Validation for Better Evaluation

In [26]:
from sklearn.model_selection import cross_val_score
tree_rmses = -cross_val_score(tree_reg, 
                              housing, housing_labels,
                              scoring="neg_root_mean_squared_error", 
                              cv=10) #k = 10 - 10 sessions
tree_rmses

array([68797.86897389, 71025.06024625, 67497.67483204, 67760.89368107,
       68170.4349214 , 67572.60730909, 67972.20945675, 66637.81518866,
       67602.27988007, 70597.14981157])

In [129]:
pd.Series(tree_rmses).describe() #validation error = 68363.399430

count       10.000000
mean     68363.399430
std       1404.275065
min      66637.815189
25%      67580.025452
50%      67866.551569
75%      68641.010461
max      71025.060246
dtype: float64

In [27]:
from sklearn.model_selection import cross_val_score 
lin_rmses = -cross_val_score(lin_reg, 
                              housing, housing_labels,
                              scoring="neg_root_mean_squared_error", 
                              cv=10)
pd.Series(lin_rmses).describe()

count       10.000000
mean     68594.058019
std       1708.582052
min      66723.018353
25%      67631.160110
50%      67976.960090
75%      69689.962275
max      71621.455299
dtype: float64

### Random Forest Regressor

In [28]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(
    preprocessing,
    RandomForestRegressor(random_state=42)
)

forest_rmses = -cross_val_score(forest_reg, housing, housing_labels, 
                                scoring="neg_root_mean_squared_error", 
                                cv=3)
forest_rmses #training error 49896.427185

array([49491.98027174, 50127.96778046, 50069.33350365])

In [29]:
pd.Series(forest_rmses).describe()

count        3.000000
mean     49896.427185
std        351.486094
min      49491.980272
25%      49780.656888
50%      50069.333504
75%      50098.650642
max      50127.967780
dtype: float64

This is a significant improvement: random forests seem very promising for this task! how about if we check the training error?

In [31]:
from sklearn.model_selection import cross_validate
forest_rmses = cross_validate(forest_reg, 
                              housing, housing_labels, 
                              scoring="neg_root_mean_squared_error", 
                              cv=2,
                              return_train_score=True)

forest_rmses

{'fit_time': array([10.62901926,  9.85702467]),
 'score_time': array([0.23313451, 0.21089268]),
 'test_score': array([-51708.37076539, -51472.64146413]),
 'train_score': array([-19333.63760917, -19027.04121217])}

- `fit_time`: The time to fit the model on the training fold for each cross 
- `test_score`: The score on the test fold for each cross-validation split.

In [32]:
forest_reg.fit(housing, housing_labels)
housing_predictions = forest_reg.predict(housing)
forest_rmse = mean_squared_error(housing_labels, 
                                 housing_predictions,
                                 squared=False)
forest_rmse

18225.232933694657

## Fine-Tune Your Model

### Grid Search

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42)),
])

full_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ocean_proximity'])])),
                ('random_forest', RandomForestRegressor(random_state=42))])

In [40]:
param_grid = [
    {'random_forest__max_features': [1, 2, 3, 4, 6, 8],
    'random_forest__max_depth':[1,2,3,4,5,6,7]}
]

grid_search = GridSearchCV(full_pipeline, param_grid, cv=2,
                           scoring='neg_root_mean_squared_error',return_train_score=True)

grid_search.fit(housing, housing_labels)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
                                                                          'housing_median_age',
                                                                          'total_rooms',
                                                                          'total_bedrooms',
                                                                          'population',
                                                                          'households',
                                                                          'median_income']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('...r',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['ocean_proximity'])])),
                                       ('random_forest',
                                        RandomForestRegressor(random_state=42))]),
             param_grid=[{'random_forest__max_depth': [1, 2, 3, 4, 5, 6, 7],
                          'random_forest__max_features': [1, 2, 3, 4, 6, 8]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [38]:
grid_search.best_params_

{'random_forest__max_depth': 7, 'random_forest__max_features': 6}

You can retrieve the best estimator by using the `grid_search.best_estimator_` attribute. If `GridSearchCV` is initialized with `refit=True` (which is the default setting), it will retrain the best estimator identified through cross-validation on the entire training set. Generally, this is beneficial as providing more data is likely to enhance its performance. 

The evaluation scores can be accessed through `grid_search.cv_results_`. This is a dictionary, but if you convert it into a DataFrame, you will have a comprehensive list of all test scores for each hyperparameter combination and for each cross-validation split, along with the mean test score across all splits.

In [39]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True) 
#[...] # change column names to fit on this page, and show rmse = -score
cv_res.head(6) # note: the 1st column is the row ID

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_random_forest__max_depth,param_random_forest__max_features,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
40,1.818943,0.000366,0.079965,0.001130,7,6,"{'random_forest__max_depth': 7, 'random_forest...",-59765.606014,-60020.519258,-59893.062636,127.456622,1
41,2.366149,0.005941,0.074119,0.000942,7,8,"{'random_forest__max_depth': 7, 'random_forest...",-59933.695907,-60042.083421,-59987.889664,54.193757,2
39,1.313214,0.009330,0.076884,0.004007,7,4,"{'random_forest__max_depth': 7, 'random_forest...",-61255.977281,-61344.158661,-61300.067971,44.090690,3
35,2.042493,0.010742,0.070782,0.000050,6,8,"{'random_forest__max_depth': 6, 'random_forest...",-62742.534869,-62714.584513,-62728.559691,13.975178,4
34,1.581483,0.004113,0.066867,0.004044,6,6,"{'random_forest__max_depth': 6, 'random_forest...",-62577.862507,-62925.114656,-62751.488582,173.626074,5
38,1.059891,0.001108,0.074467,0.003660,7,3,"{'random_forest__max_depth': 7, 'random_forest...",-63100.226855,-62968.259428,-63034.243141,65.983713,6


### Randomized Search

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'random_forest__max_features': randint(low=2, high=20)} #range

rnd_search = RandomizedSearchCV(full_pipeline, 
                                param_distributions=param_distribs, 
                                n_iter=3, #randomly take three value from the range 
                                cv=2,
                                scoring='neg_root_mean_squared_error', 
                                random_state=42
)

rnd_search.fit(housing, housing_labels)

RandomizedSearchCV(cv=2,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['longitude',
                                                                                'latitude',
                                                                                'housing_median_age',
                                                                                'total_rooms',
                                                                                'total_bedrooms',
                                                                                'population',
                                                                                'households',
                                                                                'median_income']),
                                                                              ('cat',
                                                                               Pipeline(ste...
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('onehotencoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['ocean_proximity'])])),
                                             ('random_forest',
                                              RandomForestRegressor(random_state=42))]),
                   n_iter=3,
                   param_distributions={'random_forest__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002AE729D6320>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [42]:
rnd_search.best_params_

{'random_forest__max_features': 8}

### Analyzing the Best Models and Their Errors

Frequently, examining the top-performing models can provide valuable insights into the problem at hand. For instance, the RandomForestRegressor can show the relative importance of each attribute in making precise predictions:

In [48]:
final_model = rnd_search.best_estimator_ # includes preprocessing
feature_importances = final_model["random_forest"].feature_importances_
feature_importances.round(2)

array([0.11, 0.1 , 0.05, 0.03, 0.03, 0.04, 0.02, 0.45, 0.01, 0.15, 0.  ,
       0.  , 0.01])

In [51]:
final_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                                   'median_income']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ocean_proximity'])])),
                ('random_forest',
                 RandomForestRegressor(max_features=8, random_state=42))])

In [50]:
final_model['preprocessing'].get_feature_names_out()

array(['num__longitude', 'num__latitude', 'num__housing_median_age',
       'num__total_rooms', 'num__total_bedrooms', 'num__population',
       'num__households', 'num__median_income',
       'cat__ocean_proximity_<1H OCEAN', 'cat__ocean_proximity_INLAND',
       'cat__ocean_proximity_ISLAND', 'cat__ocean_proximity_NEAR BAY',
       'cat__ocean_proximity_NEAR OCEAN'], dtype=object)

## Evaluate Your System on the Test Set

After adjusting your models over time, you finally achieve a system that performs to a satisfactory standard. You are now prepared to assess the final model using the test set. The process is straightforward: extract the predictors and labels from your test set, apply your final model to transform the data and generate predictions, then evaluate these predictions:

In [52]:
X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()

final_predictions = final_model.predict(X_test)

final_rmse = mean_squared_error(y_test, final_predictions, squared=False)
final_rmse #test error from test set

49193.87614348147

Now you're entering the pre-launch phase of the project. This stage requires you to showcase your solution, highlighting the knowledge you've gained, what was effective, what wasn't, the assumptions made, and your system's limitations. Documentation of everything is crucial, and creating engaging presentations with clear visualizations and memorable statements (e.g., "the median income is the primary predictor of housing prices") is essential. In this California housing example, the final performance of the system isn't significantly superior to the experts' price estimates, which often deviated by 30%. Nonetheless, it might still be worth launching, particularly if it allows the experts to free up time to focus on more interesting and productive tasks.

### Launch, Monitor, and Maintain Your System

Excellent, you've received the green light to launch! Now you must prepare your solution for production. This could involve refining the code, writing documentation and tests, among other tasks. Next, you can deploy your model to your production environment. The simplest method of doing this is to save the best model you've trained, transfer the file to your production setting, and load it. You can use the joblib library to save the model as follows:

In [53]:
import joblib
joblib.dump(final_model, "my_california_housing_model.pkl")

['my_california_housing_model.pkl']